In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import imutils
from imutils import face_utils
import os

In [13]:
isinMac = True
reduceResolution = True # only for thermal frame ofcourse
showRGB = True
showThermal = True
fileRGB = "P00006"
videoRGB = "rgb-r"
fileThermal = "P00006"
videoThermal = "20221212T162445"
method = 'average' # connectedcomponents or average
# code to fix the start and end time of the RGB video
startTime = (14*60)+55  # determine this time by hearing the audio in the RGB video.

# store the pixel intensity
pixelIntensity = np.array([]);


# code for RGB video
if isinMac:
    filelocationRGB = "../../../../rishiraj/Documents/thermal-project-data/cycling/Full_Study/"+fileRGB+"/"+videoRGB+".MP4"
else:
    filelocationRGB = "G:/Other computers/My MacBook Air/thermal-project-data/cycling/Full_Study/"+fileRGB+"/"+videoRGB+".MP4"
# init for face detection
print("[INFO] loading dlib thermal face detector...")
detector = dlib.get_frontal_face_detector()
# load the facial landmarks predictor
print("[INFO] loading facial landmark predictor...")
predictor = dlib.shape_predictor(os.path.join("../../calesti-data/models/shape_predictor_68_face_landmarks.dat"))
# get some details about the RGB video
captureRGB = cv2.VideoCapture(filelocationRGB)
fpsRGB = captureRGB.get(cv2.CAP_PROP_FPS)
totalFramesRGB = captureRGB.get(cv2.CAP_PROP_FRAME_COUNT)
print("Frame Rate = {} and Total Frame = {}, Length Of Video = {} s".format(fpsRGB, totalFramesRGB, totalFramesRGB/fpsRGB))
captureRGB.set(cv2.CAP_PROP_POS_FRAMES, int(startTime*fpsRGB))

# code for Thermal video
if isinMac:
    filelocationThermal = "../../../../rishiraj/Documents/thermal-project-data/cycling/Full_Study/"+fileThermal+"/"+videoThermal+".MP4"
else:
    filelocationThermal = "G:/Other computers/My MacBook Air/thermal-project-data/cycling/Full_Study/"+fileThermal+"/"+videoThermal+".MP4"
captureThermal = cv2.VideoCapture(filelocationThermal)
fpsThermal = captureThermal.get(cv2.CAP_PROP_FPS)
totalFramesThermal = captureThermal.get(cv2.CAP_PROP_FRAME_COUNT)
print("Frame Rate = {}, Total Frames = {}".format(fpsThermal, totalFramesThermal))


showRes=True;

# code to read only the first frame of the RGB video
(retRGB, frameRGB) = captureRGB.read()
frameRGB_copy = frameRGB.copy()
frameRGB = cv2.cvtColor(frameRGB, cv2.COLOR_BGRA2GRAY)


#detect face in the first frame
rects = detector(frameRGB, 0)
for rect in rects:
    (x, y, w, h) = face_utils.rect_to_bb(rect)
    cv2.rectangle(frameRGB_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)


# code for homography
# creating the SIFT algorithm
sift = cv2.SIFT_create()
# find the keypoints and descriptors with SIFT
kp_image, desc_image =sift.detectAndCompute(frameRGB[x:x+w,y:y+h], None)
# initializing the dictionary
index_params = dict(algorithm = 0, trees = 5)
search_params = dict()
# by using Flann Matcher
flann = cv2.FlannBasedMatcher(index_params, search_params)



while True:
    #(retRGB, frameRGB) = captureRGB.read()
    (retThermal, frameThermal) = captureThermal.read()
    if not retThermal:
        print("Thermal Video Ended. Program Closed")
        break;
    (hThermal, wThermal) = frameThermal.shape[:2]
    frameThermal_copy = frameThermal.copy()
    frameThermal = cv2.cvtColor(frameThermal, cv2.COLOR_BGRA2GRAY)
    
    # find the keypoints and descriptors with SIFT
    kp_grayframe, desc_grayframe = sift.detectAndCompute(frameThermal, None)
    
    # finding nearest match with KNN algorithm
    matches= flann.knnMatch(desc_image, desc_grayframe, k=2)
    
    # initialize list to keep track of only good points
    good_points=[]
    for m, n in matches:
        #append the points according
        #to distance of descriptors
        if(m.distance < 0.6*n.distance):
            good_points.append(m)
    
    # maintaining list of index of descriptors
    # in query descriptors
    query_pts = np.float32([kp_image[m.queryIdx]
                     .pt for m in good_points]).reshape(-1, 1, 2)

    # maintaining list of index of descriptors
    # in train descriptors
    train_pts = np.float32([kp_grayframe[m.trainIdx]
                     .pt for m in good_points]).reshape(-1, 1, 2)

    # finding  perspective transformation
    # between two planes
    matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)

    # ravel function returns 
    # contiguous flattened array
    matches_mask = mask.ravel().tolist()
    
    
    # initializing height and width of the image
    h, w = frameRGB[x:x+w,y:y+h].shape

    # saving all points in pts
    pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)

    # applying perspective algorithm
    dst = cv2.perspectiveTransform(pts, matrix)
    
    # using drawing function for the frame
    homography = cv2.polylines(frame, [np.int32(dst)], True, (255, 0, 0), 3)
  
    # showing the final output 
    # with homography
    cv2.imshow("Homography", homography)
            

    if showRGB:
        cv2.imshow("RBG", frameRGB_copy)
        
    key = cv2.waitKey(50)
    if key == 27 or key == ord('q'):
        break
        
    if showThermal:
        cv2.imshow("Thermal", frameThermal_copy)
    
    key = cv2.waitKey(50)
    if key == 27 or key == ord('q'):
        break

[INFO] loading dlib thermal face detector...
[INFO] loading facial landmark predictor...
Frame Rate = 8.5 and Total Frame = 20101.0, Length Of Video = 2364.823529411765 s
Frame Rate = 8.63960504662644, Total Frames = 1575.0


error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/calib3d/src/fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'findHomography'


In [11]:
totalFramesRGB

0.0